In [1]:
import os
os.environ['OPENAI_BASE_URL'] = 'https://oneapi.xty.app/v1'
os.environ["OPENAI_API_KEY"] = "sk-9QT7CG36EPZc0snHF7FfEcCf113c4d678e48C8D97fCa8cFf"

In [2]:
# attach to the same event-loop
import nest_asyncio
nest_asyncio.apply()

In [3]:
# 设置分词器：当llama_index需要分词时，会使用指定的tiktoken编码器
from llama_index.core import Settings
# tiktoken
import tiktoken
Settings.tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo").encode

/Users/skbx1060/Documents/venvs/venv_ragas/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from llama_index.core import KeywordTableIndex # 创建基于关键词的索引（通过关键词快速检索文档中的信息）
from llama_index.core import SimpleDirectoryReader # 读取位于特定目录中的文档（文档可能用于构建索引或进行文本分析）
from llama_index.llms.openai import OpenAI

In [5]:
# define LLM
llm = OpenAI(
    temperature=0,  # temperature=0 表示在生成文本时不加入随机性。在语言模型中，temperature是一个用于控制生成文本随机性的参数。temperature为 0 通常意味着模型会生成确定性最高的输出，即最可能的预测。
    model="gpt-3.5-turbo",  # 指定要使用的 OpenAI 模型的名称
    max_tokens=512 # 设置生成文本的最大令牌数。tokens是语言模型处理的文本的基本单位，通常对应于单词或字符。这个参数限制了模型一次可以生成的文本量，以避免生成过长的输出。
)

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 句子嵌入（Sentence Embeddings）是将句子转化为固定大小的向量表示，以便于进行各种NLP任务，如语义相似度比较、文本分类等。sentence-transformers 是一个流行的Python库，用于生成句子嵌入。
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-zh-v1.5"
)

In [7]:
# 内置加载器，用于从本地目录加载各种文件类型
documents = SimpleDirectoryReader("/Users/skbx1060/Documents/RAG/RAGAS/annotation_data/").load_data()

In [8]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
# parse nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

# build index
index = VectorStoreIndex(nodes,llm=llm)

In [9]:
query_engine = index.as_query_engine()

In [10]:
# get response from query
response = query_engine.query("暴发性心肌炎的定义是什么？")
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


暴发性心肌炎通常被定义为一种急骤发作的心肌炎症性疾病，其特征是伴有严重的血液动力学障碍。这一定义更侧重于临床表现，而非依赖于组织学或病理学诊断。


Evaluation

In [11]:
# import pandas as pd
# eval_df = pd.read_excel('/Users/skbx1060/Documents/RAG/RAGAS/llamaindex_ragas/Question_Context_GroundTruth.xlsx')

# eval_questions = list(eval_df['question'])
# eval_answers = list(eval_df['ground_truth'])

# eval_answers = [[a] for a in eval_answers]

In [12]:
eval_questions = [
    "暴发性心肌炎的定义是什么？",
    "UA有哪些临床表现？",
    "暴发性心肌炎在病程早期一般需要与哪些心血管系统疾病和其他可以引起相应临床表现的疾病进行区别？",
    "针对ATTR-CA的治疗分为哪些治疗？"
]

eval_answers = [
    "一般将暴发性心肌炎定义为急骤发作且伴有严重血液动力学障碍的心肌炎症性疾病",
    " UA有以下临床表现：①静息性心绞痛：心绞痛发作在休息时，并且持续时间通常在20 min以上；②初发心绞痛：1个月内新发心绞痛，可表现为自发性发作与劳力性发作并存，疼痛分级在Ⅲ级阱上；③恶化劳力型心绞痛：既往有心绞痛病史，近1个月内心绞痛恶化加重，发作次数频繁、时间延长或痛阈降低(心绞痛分级至少增加1级，或至少达到Ⅲ级)。",
    "1. 冠心病;2. 病毒性肺炎;3. 脓毒血症性心肌炎;4. 应激性心肌病（ Takotsubo综合征）;5. 普通急性心肌炎;6. 非病毒性暴发性心肌炎",
    "针对ATTR-CA的治疗分为对症治疗和病因治疗，其靶点治疗包括抑制TTR合成的药物和稳定TTR的药物。"
]

eval_answers = [[a] for a in eval_answers]

In [13]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [ ]:
from ragas.llama_index import evaluate
result = evaluate(query_engine, metrics, eval_questions, eval_answers)

In [ ]:
print(result)

In [ ]:
result.to_pandas()

In [ ]:
result_new = result.to_pandas()
# print(type(result))
# print(type(result_new))
# print(len(result))
# print(len(result_new))
# print(result_new.columns)
for i in range(len(result_new)):
    print(result_new['question'][i],':')
    print('----------------------------------------------------------------------------------')
    print('answer:---------------',result_new['answer'][i])
    print('contexts:-------------',result_new['contexts'][i])
    print('ground_truth:---------',result_new['ground_truths'][i])
    print('\n')